# Setup

In [1]:
import os
import re

import langchain
from dotenv import find_dotenv, load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings

import openai

In [2]:
langchain.__version__

'0.0.153'

In [3]:
load_dotenv(find_dotenv())

True

In [4]:
DEPLOYMENT_GPT35 = os.environ["DEPLOYMENT_MODEL_GPT35"]
DEPLOYMENT_GPT4 = os.environ["DEPLOYMENT_MODEL_GPT4"]
EMBEDDING_MODEL = "text-embedding-ada-002"

# switch between models here
DEPLOYMENT_NAME = DEPLOYMENT_GPT35

if DEPLOYMENT_NAME == DEPLOYMENT_GPT35:
    MODEL_NAME = "gpt-3.5-turbo"
elif DEPLOYMENT_NAME == DEPLOYMENT_GPT4:
    MODEL_NAME = "gpt-4"
else:
    raise ValueError(f"Unmapped model {DEPLOYMENT_MODEL}")

In [5]:
assert os.environ["OPENAI_API_TYPE"] == "azure"
assert os.environ["OPENAI_API_KEY"]
assert os.environ["OPENAI_API_VERSION"]

url_pattern = r"https://.*openai\.azure\.com/"
regex = re.compile(url_pattern)
assert bool(regex.match(os.environ["OPENAI_API_BASE"]))

In [6]:
# for chat, there is no need to set to openai's attribute, instead we pass the dict to AzureChatOpenAI
AD_CREDENTIALS = {
    "deployment_name": DEPLOYMENT_NAME,
    "openai_api_type": os.environ["OPENAI_API_TYPE"],
    "openai_api_key": os.environ["OPENAI_API_KEY"],
    "openai_api_base": os.environ["OPENAI_API_BASE"],
    "openai_api_version": os.environ["OPENAI_API_VERSION"],
}

In [7]:
# for embeddings, we need to set to openai attributes
openai.api_type = os.environ["OPENAI_API_TYPE"]
openai.api_key = os.environ["OPENAI_API_KEY"]
openai.api_version = os.environ["OPENAI_API_VERSION"]
openai.api_base = os.environ["OPENAI_API_BASE"]

# Create Chat and Embeddings

In [8]:
chat = AzureChatOpenAI(**AD_CREDENTIALS)

In [9]:
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL, chunk_size=1)